
# 한국어 텍스트 분류: TF‑IDF와 Word2Vec 실습

이 노트북에서는 간단한 한국어 문장 데이터셋을 사용하여 TF‑IDF 기반 분류기와 Word2Vec 기반 분류기를 학습해 봅니다. 데이터셋은 긍정(1)과 부정(0) 두 가지 레이블로 구성되어 있습니다. 


In [1]:

import pandas as pd

# Hugging Face 데이터셋 주소"
url = 'https://huggingface.co/datasets/Blpeng/nsmc/resolve/main/ratings.csv'
df = pd.read_csv(url)
# NSMC 데이터는 id, document, label 세 컬럼을 포함합니다.
df = df[['document', 'label']].rename(columns={'document': 'sentence'})
df =df.dropna()

In [2]:

# 간단한 전처리: 문장을 공백 기준으로 토큰화
# 한국어 형태소 분석기가 없기 때문에 공백 기준으로 나눕니다.
import re

def tokenize(sentence):
    # 특수 문자 제거하고 공백으로 분리
    sentence = re.sub(r"[^가-힣0-9\s]", "", sentence)
    tokens = sentence.strip().split()
    return tokens

# 토큰화 적용

df['tokens'] = df['sentence'].apply(tokenize)
df


,sentence,label,tokens
0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1,"[어릴때보고, 지금다시봐도, 재밌어요]"
1,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1,"[디자인을, 배우는, 학생으로, 외국디자이너와, 그들이, 일군, 전통을, 통해, 발..."
2,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1,"[폴리스스토리, 시리즈는, 1부터, 뉴까지, 버릴께, 하나도, 없음, 최고]"
3,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1,"[와, 연기가, 진짜, 개쩔구나, 지루할거라고, 생각했는데, 몰입해서, 봤다, 그래..."
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1,"[안개, 자욱한, 밤하늘에, 떠, 있는, 초승달, 같은, 영화]"
...,...,...,...
199995,포켓 몬스터 짜가 ㅡㅡ;;,0,"[포켓, 몬스터, 짜가]"
199996,쓰.레.기,0,[쓰레기]
199997,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0,"[완전, 사이코영화, 마지막은, 더욱더, 이, 영화의질을, 떨어트린다]"
199998,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0,"[왜난, 재미없었지, 라따뚜이, 보고나서, 스머프, 봐서, 그런가]"


In [3]:

# TF‑IDF 벡터화를 이용한 분류기 학습
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# 학습용과 테스트용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(df['sentence'], df['label'], test_size=0.3, random_state=42)

# TF‑IDF 벡터 생성기
vectorizer = TfidfVectorizer(tokenizer=lambda text: text.split(), min_df=1)

# 학습 데이터에 대해 벡터화 수행
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 로지스틱 회귀 분류기 학습
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

# 테스트 데이터 평가
pred = clf.predict(X_test_tfidf)
print("TF‑IDF 분류 결과:")
print(classification_report(y_test, pred))


/opt/conda/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TF‑IDF 분류 결과:
              precision    recall  f1-score   support

           0       0.77      0.82      0.80     30076
           1       0.81      0.75      0.78     29922

    accuracy                           0.79     59998
   macro avg       0.79      0.79      0.79     59998
weighted avg       0.79      0.79      0.79     59998




## Word2Vec 모델 구현

Word2Vec은 단어를 고정 길이 벡터로 표현하는 기법입니다. 대표적인 학습 방식은 **skip‑gram** 구조로, 중심 단어를 입력하여 주변 단어를 예측합니다. 여기서는 numpy를 사용하여 간단한 skip‑gram 모델을 구현합니다. 학습이 간단한 만큼 데이터셋도 작은 규모로 진행합니다.


In [12]:
# ==== Word2Vec with gensim, doc embeddings, classifier, visualization ====
import os
import re
import numpy as np
import pandas as pd
from collections import Counter

from gensim.models import Word2Vec, KeyedVectors

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import random

# 재현성 고정
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

sentences = df['tokens'].values.tolist()

# 2. Word2Vec 학습
# sg=0는 CBOW, sg=1은 Skip-gram
w2v = Word2Vec(
    sentences=sentences,
    vector_size=100,
    window=8,
    min_count=5,
    workers=4,
    sg=0,
    epochs=5  # 필요 시 늘리세요
)

# 3. vocabulary 확인
vocab = list(w2v.wv.key_to_index.keys())
print(f"단어 수: {len(vocab)}")
print("예시 단어 20개:", vocab[:20])

# 4. 모델 저장과 로드 예시
# w2v.save("w2v_model.model")
# 키드벡터만 저장하려면
# w2v.wv.save("w2v_vectors.kv")

# 필요 시 다시 로드
# w2v = Word2Vec.load("w2v_model.model")
# kv = KeyedVectors.load("w2v_vectors.kv")

단어 수: 29984
예시 단어 20개: ['영화', '너무', '정말', '진짜', '이', '왜', '그냥', '더', '이런', '수', '영화를', '잘', '다', '보고', '좀', '영화는', '영화가', '그', '본', '최고의']


In [17]:

# 5. 문서 임베딩 만들기
# 각 문서의 토큰 중 vocab에 존재하는 단어 벡터 평균
def doc_vector(tokens, model):
    vecs = [model.wv[w] for w in tokens if w in model.wv]
    if len(vecs) == 0:
        return np.zeros(model.vector_size, dtype=np.float32)
    return np.mean(vecs, axis=0)

X = np.vstack([doc_vector(toks, w2v) for toks in sentences])

# 레이블 y 준비. 이미 df['label']이 0,1 형태라면 그대로 사용
if 'label' in df.columns:
    y = df['label'].values
else:
    # 레이블이 없으면 모두 0으로 세팅. 실제 실험에서는 반드시 레이블을 제공하세요.
    y = np.zeros(len(df), dtype=int)

# 6. 분류기 학습과 평가
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y if len(np.unique(y)) > 1 else None
)

clf = LogisticRegression(
    max_iter=200,
    n_jobs=None  # 최신 sklearn에서는 n_jobs가 제거된 경우가 있어 옵션 생략 권장
)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))


Accuracy: 0.6863171579289482
              precision    recall  f1-score   support

           0       0.67      0.74      0.70     20000
           1       0.71      0.63      0.67     19999

    accuracy                           0.69     39999
   macro avg       0.69      0.69      0.69     39999
weighted avg       0.69      0.69      0.69     39999



/opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
